### Toy Network Training

Below, we train a toy neural network and optimize its space using `reducible`

In [252]:
%load_ext autoreload
%autoreload 2

# toy network example 
import tensorflow as tf
import numpy as np

mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

base_model = tf.keras.models.Sequential()
base_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
base_model.add(tf.keras.layers.Dense(500, activation='relu'))
base_model.add(tf.keras.layers.Dense(500, activation='relu'))
base_model.add(tf.keras.layers.Dropout(0.2))
base_model.add(tf.keras.layers.Dense(10))

base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
base_model.fit(x_train, y_train, epochs=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1875/1875 [==============================] - 25s 12ms/step - loss: 0.4728 - accuracy: 0.8293


### after training, our original model gets ~85% accuracy

In [253]:
acc_base = base_model.evaluate(x_test,  y_test, verbose=2)[1]

313/313 - 1s - loss: 0.4070 - accuracy: 0.8538 - 1s/epoch - 4ms/step


In [254]:
from reducible.optimize import optimize_model

optimized_model = optimize_model(base_model, x_test, y_test, opt_factor=0.05, MAX_ITER=15)

313/313 - 2s - loss: 1.2531 - accuracy: 0.4818 - 2s/epoch - 7ms/step
313/313 - 2s - loss: 1.2529 - accuracy: 0.4816 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 1.2529 - accuracy: 0.4821 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.9188 - accuracy: 0.6209 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.9186 - accuracy: 0.6212 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.9184 - accuracy: 0.6210 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.6630 - accuracy: 0.7916 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.6629 - accuracy: 0.7919 - 2s/epoch - 5ms/step
313/313 - 2s - loss: 0.6619 - accuracy: 0.7940 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.6120 - accuracy: 0.8035 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.6121 - accuracy: 0.8035 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.6113 - accuracy: 0.8042 - 2s/epoch - 5ms/step
313/313 - 2s - loss: 0.5125 - accuracy: 0.8310 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.5127 - accuracy: 0.8304 - 2s/epoch - 6ms/step
313/313 - 2s - loss: 0.5126 - accu

### and our parameter-reduced model gets ~85%!

In [255]:
acc_opt = optimized_model.evaluate(x_test,  y_test, verbose=2)[1]

313/313 - 2s - loss: 0.4587 - accuracy: 0.8500 - 2s/epoch - 6ms/step


### Calculating the percent difference in accuracy

In [263]:
100*(acc_base - acc_opt)/acc_base

0.4450661658331348

### Saving our model to disk

In [257]:
from reducible.save import save_model_to_file, load_model_from_file

save_model_to_file(optimized_model, "optimized_model.gyatt")
tf.keras.models.save_model(base_model, "base_model.keras")

In [258]:
loaded_model = load_model_from_file("optimized_model.gyatt")

In [259]:
# checking to make sure model still works 

loaded_model.evaluate(x_test,  y_test, verbose=2);

313/313 - 2s - loss: 0.4587 - accuracy: 0.8500 - 2s/epoch - 5ms/step


## Future improvements: include serialization optimizations to close the gap

In [261]:
opt_total = 0
base_total = 0
# calculates predicted reduction by simply calculating the number of terms in weights and bias matrices 

from reducible.optimize import RankKApprox

for layer in optimized_model.layers: 
    if isinstance(layer, RankKApprox):
        opt_total += layer.A.numpy().size + layer.B.numpy().size +  layer.b.numpy().size

for layer in base_model.layers: 
    if layer.get_weights(): 
        w, b = layer.get_weights()
        opt_total += w.size
        opt_total += b.size

predicted_reduction = 1-opt_total/base_total

In [260]:
import os
actual_reduction = 1-os.path.getsize('optimized_model.gyatt')/os.path.getsize('base_model.keras')

In [262]:
print(f"Predicted reduction {predicted_reduction*100:0.1f}% | Actual reduction {actual_reduction*100:0.1f}%")

Predicted reduction 65.9% | Actual reduction 16.1%
